# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [2]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

input_path = './Data/Iris_inputs.xlsx'
input_sheet = 'Iris'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_Iris.xlsx'

iterations = 10000
chunks = 'auto'  #'auto' or more than 500

In [3]:
from Tools_Iris import *
from Model_Iris import *

# LCI

The aircraft's life cycle was divided into four parts:
1. Development and Engineering
2. Manufacturing and Assembly
3. Operation
4. End-of-Life

First, we execute function to read inputs from designated `input_path` and transform it into Parameters and ParameterSet object `p`.

In [3]:
inputs = read_inputs(input_path, input_sheet)

In [4]:
p = ParameterSet(inputs, iterations, chunks)

Samples are stored in a xr.Dataset object accessible throught the `p.data` ParameterSet attribute.

Then, we run the function that calculates the functional unit for the study.

In [5]:
p = func_unit(p)

In [6]:
p.data["ha_fleet"].mean().compute()

<xarray.DataArray 'ha_fleet' ()>
array(2.40031708e+09)

In [7]:
p.data["ha_life"].mean().compute()

<xarray.DataArray 'ha_life' ()>
array(5690865.44756203)

In [10]:
p.data

<xarray.Dataset>
Dimensions:             (i: 1000)
Coordinates:
  * i                   (i) int32 0 1 2 3 4 5 6 ... 993 994 995 996 997 998 999
Data variables: (12/83)
    takt                (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    OEW                 (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    flights_year        (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    FH                  (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    productivity        (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    lifetime            (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    ...                  ...
    p_recycl_GFRP       (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    p_recycl_CFRP       (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    ha_flight           (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    ha_year             (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    ha_life             (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    ha_fleet            (i) float64 dask.array<chunksize=(1000,), meta=np.ndarray>

<br></br>
Executing the UP reading and writing functions.

In [11]:
UP_dataframe = read_unit_processes(database_path)

In [12]:
UP = unit_process_dataset(UP_dataframe)

Building the empty LCI to start the calculation.

In [13]:
inventory = LCI(name=input_sheet, iterations=iterations, UP=UP, parameters=p)

In [14]:
inventory.run()

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 1000)
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
  * i               (i) int32 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
Data variables: (12/16)
    Office          (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Infrastructure  (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Capital         (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Materials       (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Factory         (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Logistics       (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    ...              ...
    Fuel            (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Recycling       (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Incineration    (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Landfill        (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Prototypes      (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
    Certification   (Substances, i) float64 dask.array<chunksize=(1835, 1000), meta=np.ndarray>
Attributes:
    Name:     Iris

The LCI data can be accessed by the `LCI.data` attribute.

# LCIA

Extracting the characterization factors from the excel file located in `database_path`.

In [15]:
MP_data, EP_data = read_CF(database_path)

Creating an instance of the characterization factors class:

In [16]:
CF = CharactFactors(MP_data, EP_data, UP.Substances)

Adjusting the NLT category CFs.

In [17]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

The impacts CFs can be accessed by the `CharactFactors.Midpoint.dataset` or `CharactFactors.Endpoint.dataset` attributes.

In [18]:
display(CF.MP.dataset)
display(CF.EP.dataset)

<xarray.Dataset>
Dimensions:         (Substances: 1835)
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
Data variables: (12/18)
    FE              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MRD             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    POF             (Substances) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MET             (Substances) float64 0.0 0.0 0.000283 ... 0.0 0.0 0.0
    TA              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ULO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...              ...
    ALO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    HT              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    IR              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CC              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

<xarray.Dataset>
Dimensions:     (Categories: 18)
Coordinates:
  * Categories  (Categories) object 'CC' 'OD' 'TA' 'FE' ... 'WD' 'MRD' 'FD'
Data variables:
    HH          (Categories) float64 1.4e-06 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ED          (Categories) float64 7.93e-09 0.0 5.8e-09 ... 0.0 0.0 0.0
    RA          (Categories) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0715 0.165

<br></br>
**Executing the Impact Assessment.**

In [19]:
aeco = LCIA.build(inventory, CF)

## Midpoint Results

Raw midpoint results can be accessed by the `LCIA.MP` attribute. Mean midpoint results are calculated by the `LCIA.mean` method, with pathway set to "MP".

In [20]:
display(aeco.MP)

<xarray.Dataset>
Dimensions:         (Categories: 18, i: 1000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * Categories      (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'FD' 'WD' 'CC'
Data variables: (12/16)
    Office          (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Infrastructure  (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Capital         (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Materials       (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Factory         (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Logistics       (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    ...              ...
    Fuel            (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Recycling       (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Incineration    (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Landfill        (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Prototypes      (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
    Certification   (i, Categories) float64 dask.array<chunksize=(1000, 18), meta=np.ndarray>
Attributes:
    Name:     Iris

In [26]:
aeco.mean("MP", by='sum').compute()

<xarray.DataArray 'stack-5d83fa2d0bfe51b1c0a10adcd10a6b35' (Categories: 18)>
array([3.73071113e-02, 9.03647833e+00, 2.22157023e-01, 1.72979313e+01,
       3.88596252e-01, 3.71000557e-01, 1.72405367e-01, 7.32091664e-02,
       1.02515793e+00, 2.81953007e-03, 1.24489034e-04, 2.70868515e+00,
       8.90882632e+00, 3.89362868e+02, 7.71624402e-01, 2.48652663e+01,
       5.22510307e+01, 5.88248781e+01])
Coordinates:
  * Categories  (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'IR' 'FD' 'WD' 'CC'

## Endpoint Results

Raw endpoint results can be accessed by the `LCIA.EP` attribute. Mean endpoint results are calculated by the `LCIA.mean` method, with pathway set to "EP".

In [22]:
display(aeco.EP)

<xarray.Dataset>
Dimensions:         (AOP: 3, i: 1000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * AOP             (AOP) <U2 'HH' 'ED' 'RA'
Data variables: (12/16)
    Office          (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Infrastructure  (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Capital         (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Materials       (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Factory         (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Logistics       (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    ...              ...
    Fuel            (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Recycling       (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Incineration    (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Landfill        (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Prototypes      (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
    Certification   (i, AOP) float64 dask.array<chunksize=(1000, 3), meta=np.ndarray>
Attributes:
    Name:     Iris

In [24]:
aeco.mean("EP", by="sum").compute()

<xarray.DataArray 'stack-612c3cd7be455dca013744b10c410262' (AOP: 3)>
array([2.83707451e-04, 4.83571325e-07, 4.74229246e+00])
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

## Contribution to Variance (CTV)

Calculate CTV results for all inputs with midpoint and endpoint results. CTV results can be acessed by the `LCIA.CTV` attribute.

In [27]:
aeco.build_CTV(parameterset=p)

<xarray.Dataset>
Dimensions:     (Parameters: 79)
Coordinates:
  * Parameters  (Parameters) <U18 'takt' 'OEW' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MRD         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    POF         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MET         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    TA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ULO         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...          ...
    FD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    WD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    CC          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    HH          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ED          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    RA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>

In [28]:
aeco.CTV

<xarray.Dataset>
Dimensions:     (Parameters: 79)
Coordinates:
  * Parameters  (Parameters) <U18 'takt' 'OEW' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MRD         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    POF         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MET         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    TA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ULO         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...          ...
    FD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    WD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    CC          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    HH          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ED          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    RA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>

In [30]:
aeco.CTV["CC"].sortby(aeco.CTV["CC"], ascending=False).compute()

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Iris.py:307: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))
c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Iris.py:307: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


<xarray.DataArray 'CC' (Parameters: 79)>
array([4.82275847e+01, 4.48851901e+01, 4.24590817e-01, 4.01830772e-01,
       3.78733360e-01, 3.71614139e-01, 3.43695455e-01, 3.19671994e-01,
       3.15922201e-01, 2.85808821e-01, 2.75730063e-01, 2.71838508e-01,
       2.57670300e-01, 2.36202994e-01, 2.27606556e-01, 2.24735536e-01,
       2.05980383e-01, 1.91061178e-01, 1.78793314e-01, 1.71637550e-01,
       1.49759396e-01, 1.04500070e-01, 1.01733363e-01, 9.58933646e-02,
       8.67334270e-02, 8.48616807e-02, 8.30347412e-02, 7.64937762e-02,
       7.15127965e-02, 6.26376777e-02, 6.21420680e-02, 6.21380012e-02,
       5.82182161e-02, 5.43351499e-02, 5.27438313e-02, 5.18659460e-02,
       4.58054058e-02, 4.38172059e-02, 4.24937740e-02, 3.76783073e-02,
       3.76007850e-02, 3.34503534e-02, 3.22869404e-02, 3.17202404e-02,
       2.87795317e-02, 2.59109317e-02, 2.46115194e-02, 2.27464471e-02,
       1.80958882e-02, 1.71502320e-02, 1.54352013e-02, 1.33812890e-02,
       1.22628188e-02, 1.16324872e-02, 1.00036406e-02, 9.99950264e-03,
       7.57828305e-03, 6.81868151e-03, 3.38882717e-03, 2.81221394e-03,
       1.84867472e-03, 1.55360764e-03, 1.50961898e-03, 1.40228716e-03,
       1.33493180e-03, 1.21778535e-03, 8.57502343e-04, 2.91026830e-04,
       2.18289145e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])
Coordinates:
  * Parameters  (Parameters) <U18 'pesticide_eff' ... 'ironbirds'

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [31]:
aeco.save(output_path, LCI=False)

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Iris.py:307: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


LCIA saved at ./Outputs/Iris_outputs


In [22]:
#aeco.to_excel(output_path)